In [1]:
import dask_mongo
from dask.distributed import Client, LocalCluster, futures_of, as_completed
from dask import array as da
import numpy as np
import pandas as pd
import logging
import os
import pathlib
import sys

from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from utils import resize
from pillow_heif import register_heif_opener

from tqdm import tqdm

register_heif_opener()

In [2]:
from utils import rotate_image

In [3]:
sys.path.append(pathlib.Path(os.getcwd(), 'deepface').as_posix())

In [4]:
from deepface.DeepFace import dst

2023-03-11 23:16:46.420669: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 23:16:46.914871: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/anuvini/mambaforge/envs/aialbum/lib/python3.10/site-packages/cv2/../../lib64::/usr/local/cuda/lib64/
2023-03-11 23:16:46.914919: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/anuvini/mambaforg

In [5]:
# cluster = LocalCUDAClusterdistributedlence_logs=logging.ERROR)
cluster = LocalCluster(n_workers=32, threads_per_worker=1)
client = Client(cluster)

model = 'ArcFace'
detector = "retinaface"
distance = "euclidean"

In [6]:
def show_image(path, face):
    img = Image.open(f'/media/anuvini/Anuradha WD/Family/{path}').convert('RGB')
    img = rotate_image(img)
    img_bb = ImageDraw.Draw(img)
    print(face)
    (top, right, bottom, left) = face['top'], face['right'], face['bottom'], face['left']
    shape = [left, top, right, bottom]
    img_bb.rectangle(shape, outline="red", width=20)
    ratio, img = resize(img, 500, 500)
    return img

In [7]:
client.cluster

LocalCluster(87a6caac, 'tcp://127.0.0.1:33989', workers=32, threads=32, memory=125.74 GiB)

In [8]:
bag = dask_mongo.read_mongo(
    database="ai_album",
    collection="media",
    connection_kwargs={
        "host": "localhost", 
        "port": 27017,
        "username": "aialbumadmin",
        "password": "aialbumpw"
    },
    chunksize=10,
)

In [9]:
dst.findThreshold(model, distance)

4.15

In [10]:
def has_faces(x):
    if 'faceEncodings' in x and len(x['faceEncodings']) > 0:
        return True
    
    return False

def as_df(x):
    arr = []
    
    for fid, enc in enumerate(x['faceEncodings']):
        dct_meta = dict()
        dct_meta['_id'] = str(x['_id'])
        dct_meta['face'] = x['faces'][fid]
        dct_meta['enc'] = enc
        dct_meta['path'] = x['path']
        arr.append(dct_meta)
    
    return arr
        
with_faces = bag.filter(has_faces)
formatted = with_faces.map(as_df)
flattened = formatted.flatten()
flattened = flattened.repartition(npartitions=32)
flattened = flattened.persist()

In [11]:
def get_matching(this, that):
    dist = dst.findEuclideanDistance(this['enc'], that['enc'])                
    # if deviant, skip group
    if dist > dst.findThreshold(model, distance):
        return None
    return this

def get_mismatching(this, that):
    dist = dst.findEuclideanDistance(this['enc'], that['enc'])                
    # if deviant, skip group
    if dist <= dst.findThreshold(model, distance):
        return None
    return this

In [12]:
!rm -rf face_groups

In [13]:
scan = True
source = flattened
group = 1

pbar = tqdm(source.count().compute())

while source.count().compute() > 0:    
    items = source.take(1, npartitions=source.npartitions)
    item = items[0]

    matching = source.map(get_matching, item)
    matching = matching.filter(lambda x: x is not None)
    matches = matching.count().compute()

    mismatching = source.map(get_mismatching, item)
    mismatching = mismatching.filter(lambda x: x is not None)
            
    # non unique person
    if matches > 50:
        # print('Matching', matches)
        # print('Mismatching', mismatches)
        matching = matching.map(str)
        matching.to_textfiles(f'./face_groups/{group}')
        group += 1

    pbar.update(matches)
    source = mismatching.repartition(npartitions=32)
    source = source.persist()
    


1212it [00:13, 31.78it/s] 

In [14]:
# https://keestalkstech.com/2020/05/plotting-a-grid-of-pil-images-in-jupyter/

def display_images(
    images: [Image], 
    columns=5, width=20, height=8, max_images=15, 
    label_wrap_length=50, label_font_size=8):

    if not images:
        print("No images to display.")
        return 

    if len(images) > max_images:
        print(f"Showing {max_images} images of {len(images)}:")
        images=images[0:max_images]

    height = max(height, int(len(images)/columns) * height)
    plt.figure(figsize=(width, height))
    for i, image in enumerate(images):

        plt.subplot(int(len(images) / columns + 1), columns, i + 1)
        plt.imshow(image)

        if hasattr(image, 'filename'):
            title=image.filename
            if title.endswith("/"): title = title[0:-1]
            title=os.path.basename(title)
            title=textwrap.wrap(title, label_wrap_length)
            title="\n".join(title)
            plt.title(title, fontsize=label_font_size)

In [15]:
images = [show_image(matching[0]['path'], matching[0]['face'])]

for g_images in mismatching[:10]:
    images.append(show_image(g_images['path'], g_images['face']))
    
    

TypeError: 'Bag' object is not subscriptable

In [ ]:
display_images(images, columns=3, max_images=100)

In [ ]:
import gc
gc.collect()

1223it [00:29, 31.78it/s]